In [1]:
#add necessary libraries
import networkx as nx #library supporting networks
import matplotlib.pyplot as plt #plotting
import pandas as pd
import numpy as np
import scipy.stats as stat
from scipy import optimize
#make sure plots are embedded into the notebook
%pylab inline 
import statsmodels.formula.api as smf
#import shapefile as shp
#from shapely.geometry.polygon import Polygon
from descartes import PolygonPatch
import os
from networkx.algorithms import community
from sklearn.mixture import GaussianMixture 

# Import required libraries
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Dropout
from keras import regularizers
from keras import optimizers
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import set_random_seed
from keras import backend as K
import datetime
from sklearn.mixture import GaussianMixture
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import operator
import json

%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [6]:
RecordWritingPath = '/Users/hemingyi/Documents/capstone/Synthetic_data/'
DataPath = '/Users/hemingyi/Documents/capstone/Synthetic_data/'
comboPath = '/Users/hemingyi/Documents/capstone/combo/'


In [ ]:
columnInfo = pd.read_csv('columnInfo.csv')
columnInfo['generated'] = columnInfo['Unnamed: 0'].apply(lambda x: 'station' + str(x))

In [ ]:
df = columnInfo.append([columnInfo]*9999,ignore_index=True)

In [ ]:
synNP = np.load(DataPath+'syntheticGlobalAnomaly.npy')

In [ ]:
syn = pd.DataFrame(data=synNP)
syn['date'] = syn.index
name = columnInfo['generated'].tolist() + ['date']
syn.columns = name

In [ ]:
# "annually" data
ann = syn.sum()
ann = pd.DataFrame(data=ann)

In [ ]:
TransformAnn.head()

In [ ]:
ann['station'] = ann.index
ann.columns = ['amount','station']
ann = ann[['station','amount']]

In [ ]:
TransformAnn = ann.merge(columnInfo,left_on='station',right_on='generated')

In [ ]:
## transform to the transportation data format
transform = pd.wide_to_long(syn, ['station'], i="date", j="stationame")

In [ ]:
transformDF = transform.merge(columnInfo,left_on='stationame',right_on='Unnamed: 0')

In [ ]:
TransformAnn = TransformAnn[['amount','start_id','end_id']]
TransformAnn.to_csv('TaipeiAnnuallySynthetic.csv',header=True)

In [ ]:
transformDF = transformDF[['station','start_id','end_id']]
transformDF['date'] = transformDF.index
transformDF.columns = ['amount','start_id','end_id','date']

In [ ]:
transformDF.to_csv('TaipeiDateWiseSynthetic.csv',header=True)

In [ ]:
#events

In [2]:
def readTransDF(TransportationDataPath,city):
    df = pd.read_csv(TransportationDataPath+city+'EdgeYearwiseAggregated.csv')
    return df
def makeGraphfromDf(df):
    G=nx.DiGraph()
    nx.set_edge_attributes(G,'weight', 0)
    for k in df.index:
        G.add_edge(df['start_id'][k],df['end_id'][k],weight=df['amount'][k])
#     nx.write_edgelist(G, comboPath+'temp/%s.net'%city)
    return G

In [3]:
def getComboPartition(G,comboPath,city):
    #save network in net format
    nodenum={}
#     G = makeGraphfromDf(df)
    #create a dictionary transforming nodes to unique numbers
    nodes = list(G.nodes())
    print('nodes amount: ',len(nodes))
    for i,j in enumerate(list(G.nodes())):
        nodenum[str(j)] = str(i)
#         nodes[str(i)] = str(j)
#     i=0
#     for n in list(G.nodes()):
#         nodenum[n]=i
#         nodes[i]=n
#         i+=1

    tempNetFile = comboPath+'temp/%s.net'%city
    f = open(tempNetFile, 'w')
    f.write('*Arcs\n')

    for e in G.edges(data=True):
        f.write('{0} {1} {2}\n'.format(nodenum[str(e[0])],nodenum[str(e[1])],e[2]['weight']))
    f.close()

    #run combo
    command= comboPath+'/comboCPP '+tempNetFile#+' '+str(maxcom)
    os.system(command)

    #read resulting partition
    partitionFile = comboPath+'temp/'+city + '_comm_comboC++.txt'
    f = open(partitionFile, 'r')
    i=0
    partition={}
    for line in f:
        partition[str(nodes[i])]=str(int(line))
        i+=1
#         print(i)
    f.close()
    os.remove(partitionFile) 
    os.remove(tempNetFile)
#     print("Communities: ",len(set(partition.values())))
    return partition 

In [18]:
def getSubCommunity(partition,df,city):
    df['start_community'] = df['start_id'].apply(lambda x: partition[x])
    df['end_community'] = df['end_id'].apply(lambda x: partition[x])
    data2 = df[df.start_community == df.end_community]
    communities = list(set(df.start_community))
    print('communities numbers: ',communities)
    SubPartition = {}
    for c in communities:
        print('detecting sub-communities in community ',c)
        d = data2[data2.start_community == c]
        graph = makeGraphfromDf(d)
        p = getComboPartition(graph,comboPath,city)
        SubPartition[c] = p
    return (SubPartition,df)

In [16]:
df['start_community'] = df['start_id'].apply(lambda x: partition[str(x)])

In [19]:
df = pd.read_csv('TaipeiAnnuallySynthetic.csv')
G = makeGraphfromDf(df)
partition = getComboPartition(G,comboPath,'TaipeiSyn')
SubPartition,df = getSubCommunity(partition, df,'TaipeiSyn')

nodes amount:  108
communities numbers:  ['1', '3', '2', '0']
detecting sub-communities in community  1
nodes amount:  23
detecting sub-communities in community  3
nodes amount:  21
detecting sub-communities in community  2
nodes amount:  40
detecting sub-communities in community  0
nodes amount:  24


In [22]:
import json

with open('partition.json', 'w') as fp:
    json.dump(partition, fp)

with open('SubPartition.json', 'w') as fp:
    json.dump(SubPartition, fp)


In [9]:
def aggregateByCommunities(partition,SubPartition,data):


    print('Raw shape: ',data.shape)
    if 'Date' in data.columns:
        data['date'] = pd.to_datetime(data.Date)
    elif 'date' in data.columns:
        data['date'] = pd.to_datetime(data.date)
    else:
        print(data.columns)

    data['start_community'] = data['start_id'].apply(lambda x: partition[str(x)])
    data['end_community'] = data['end_id'].apply(lambda x: partition[str(x)])
    data['subStart'] = data.apply(lambda x: str(x['start_community'])+'.'+str(SubPartition[x['start_community']][str(x['start_id'])]),axis=1)
    data['subEnd'] = data.apply(lambda x: str(x['end_community'])+'.'+str(SubPartition[x['end_community']][str(x['end_id'])]),axis=1)

    communityData = data[['subStart','subEnd','date','amount']]
    communityData = communityData.groupby(['subStart','subEnd','date']).sum().reset_index()
    communityData.columns = ['start_id', 'end_id', 'date', 'amount']

    return communityData

In [10]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='amount', index=['date'],
                    columns=['start_id','end_id'], aggfunc=np.sum, fill_value=0)
    return table

In [11]:
def anomalyDetection(y,pval = 0.2,iterN=20):
    #index of regular (non-outlier points)

    rind = np.array(range(y.shape[0]))
    
    #clustering model
    gm=GaussianMixture(n_components=3,n_init=100,max_iter=1000,random_state=0) 
    for i in range(iterN): #iterate
        print('Iteration {}'.format(i+1))  
        clustering=gm.fit(y[rind,:]) #fit EM clustering model excluding outliers
        l=clustering.score_samples(y) #estimate likelihood for each point
        Lthres=sorted(l)[int(len(l)*pval)] #anomaly threshold
        rind0=0+rind
        rind=l>Lthres #non-anomalous points
        if all(rind==rind0):
            print('Convergence in {} iterations'.format(i+1))
            break
    return l < Lthres

In [12]:
# import events data
def getEvents(EventDataPath,city):
    events_data =EventDataPath+city+'Events.csv'
    df_events = pd.read_csv(events_data, encoding = "ISO-8859-1", parse_dates=['Date'], infer_datetime_format=True)

    # dataframe for events
    df_finalEvents =  df_events[['Date', 'Type']]

    # list events666
    
    lis_event = df_finalEvents['Type'].unique()
    lis_event = list(lis_event)
    return (lis_event,df_finalEvents)

In [13]:
city = 'SyntheticTaipei'

In [14]:
events = [5048, 2337, 7935, 9157, 9339, 5923, 6570, 7134, 6580, 3196, 4943, 7458, 2159, 2482, 1401, 463, 8227, 4553, 1028, 4324, 1851, 8351, 3047, 697, 628, 1332, 9297, 6616, 4510, 9871, 8898, 1765, 7267, 7310, 4005, 2731, 6373, 615, 681, 9190, 1897, 9724, 6830, 3523, 913, 5043, 4290, 385, 6822, 1512, 7902, 3078, 6333, 9577, 77, 5236, 6593, 8246, 7518, 5064, 642, 6995, 5525, 4289, 1595, 4662, 4731, 9146, 4043, 8050, 8411, 8185, 3985, 2837, 7087, 1950, 7912, 6896, 6398, 6275, 2051, 3488, 2311, 8225, 3600, 3002, 9089, 9615, 8664, 1018, 8897, 6234, 6806, 4081, 9598, 7213, 6181, 705, 394, 1618, 347, 7793, 4608, 7602, 5636, 3988, 5049, 3004, 6924, 5280, 6092, 1656, 7878, 2991, 9848, 7296, 7112, 6365, 7648, 3940, 3588, 9760, 7251, 6205, 9822, 447, 9163, 9115, 2855, 7120, 2236, 9448, 591, 6108, 4265, 3098, 3522, 4398, 4338, 870, 8727, 7630, 2416, 4048, 7381, 4225, 1214, 7262, 5538, 8622, 7606, 7736, 9863, 9400, 3305, 126, 5660, 1494, 3743, 5434, 7050, 9564, 6048, 9298, 3921, 4968, 1404, 9508, 2096, 856, 8576, 4220, 7188, 2457, 477, 4625, 747, 4055, 3694, 3236, 6164, 6938, 5747, 2981, 5162, 8252, 3740, 4932, 2196, 9426, 9904, 109, 5922, 5099, 1551, 8907, 4232, 4184, 9000, 99, 2958, 730, 7864, 848, 5871, 5804, 1002, 3263, 9165, 9902, 1593, 5704, 9418, 4769, 7239, 9859, 4847, 9119, 1819, 6416, 6019, 3310, 1784, 4876, 5542, 8113, 7473, 344, 1643, 3461, 5347, 5159, 1955, 8947, 589, 9771, 2504, 1884, 216, 2149, 871, 801, 6413, 7943, 7330, 4560, 7475, 7196, 5981, 2305, 4777, 860, 9414, 1399, 3430, 1496, 2561, 6689, 4915, 2908, 8833, 3369, 8955, 8319, 1833, 4877, 7249, 148, 4688, 4092, 3998, 6539, 4969, 1647, 3966, 1381, 4296, 5314, 6039, 5326, 843, 896, 5832, 1216, 2544, 7701, 6473, 2722, 6392, 6489, 7268, 3502, 5932, 6509, 2604, 2700, 8450, 7447, 8788, 1564, 4278, 3160, 9466, 4079, 6285, 1761, 5419, 4064, 4776, 9838, 3786, 104, 2299, 4991, 932, 6236, 9527, 4617, 3325, 7929, 5941, 8322, 4727, 2910, 2372, 2820, 5607, 5661, 9488, 4887, 7107, 8163, 9120, 6468, 6228, 190, 5176, 6745, 5900, 6606, 2015, 5005, 3515, 6334, 8754, 4136, 4787, 8781, 2934, 3862, 909, 9128, 4535, 5905, 757, 6573, 127, 8666, 1375, 6563, 7571, 3480, 4363, 929, 6902, 2798, 7456, 6831, 9750, 5379, 865, 9670, 9836, 5393, 9230, 93, 9443, 781, 6245, 3438, 2992, 7796, 9735, 1268, 1527, 3738, 895, 3517, 5447, 4365, 6319, 696, 6369, 4916, 188, 265, 8957, 7953, 9464, 2469, 6451, 5999, 9559, 1032, 6854, 8211, 9736, 9755, 4201, 867, 9923, 8303, 7365, 2523, 9725, 4957, 3005, 8308, 8334, 7579, 7115, 8518, 6377, 1918, 4687, 4567, 7044, 4442, 5554, 1240, 7109, 6362, 1286, 1693, 8, 3886, 6322, 3073, 3701, 209, 5220, 2580, 2499, 4268, 7375, 7082, 304, 5523, 8595, 3378, 7823, 8601, 3635, 926, 5545, 1164, 5238, 8786, 4578, 7233, 6132, 302, 9850, 3734, 9324, 4045, 5125, 382, 105, 2518, 1518, 6566, 5062, 8102, 9654, 2248, 2277, 4461, 7567, 529, 4359, 5155, 3061, 2094, 1483, 5387, 7246, 9392, 1034, 4387, 1283, 7916, 4529, 7913, 2192, 3009, 9396, 75, 4914, 5611]
EventDFTrans = pd.DataFrame()
EventDFTrans['Date'] = events

In [ ]:
f = open(RecordWritingPath+'F1ScoreSubCommunity.txt', 'w+')
f.write('Sub Community Detection\n')
f.write('Taipei Synthetic Global Events'+'\n')

df = pd.read_csv('TaipeiAnnuallySynthetic.csv')
# lis_event,df_finalEvents = getEvents(EventDataPath,city)
df_finalEvents = EventDFTrans
G = makeGraphfromDf(df)
partition = getComboPartition(G,comboPath,city)
SubPartition,df = getSubCommunity(partition,df)
datewiseDF = pd.read_csv('TaipeiDateWiseSynthetic.csv')
data = aggregateByCommunities(partition,SubPartition,data=datewiseDF)
dataTs = getTimeSeries(data)
matrix = dataTs.values
matrix = np.log(matrix+1)
for i in range(matrix.shape[1]):
    matrix[:, i] = (matrix[:, i] - matrix[:, i].min()) / (matrix[:, i].max() - matrix[:, i].min())
date = dataTs.index.to_frame().rename(columns={'date':'Date'})
result = {}
EventsDF = df_finalEvents['Date'].drop_duplicates().to_frame()
EventsDF['Anomaly'] = True
df = EventsDF.merge(date,on='Date',how='right')
for thres in range(1,10, 1):
    th = thres/10
    print("Threshhold: ",th)
    outliers = anomalyDetection(matrix,pval = th)
    df['outlier'] = outliers
    TP = len(df[(df['outlier']==True)&(df['Anomaly']==True)])
    FP = len(df[(df['outlier']==False)&(df['Anomaly']==True)])
    TN = len(df[(df['outlier']==False)&(df['Anomaly']==False)])
    FN = len(df[(df['outlier']==False)&(df['Anomaly']==True)])

    Precision = len(df[(df['outlier']==True)&(df['Anomaly']==True)])/len(df[df['outlier']==True])
    Recall = len(df[(df['outlier']==True)&(df['Anomaly']==True)])/len(EventsDF)
    if Precision+Recall > 0:
        F1 = (2*Precision*Recall)/(Precision+Recall)
    else:
        F1 = 0
    result[th] = F1
F1df = pd.DataFrame.from_dict(data)
F1df['city'] = city
F1df.to_csv(RecordWritingPath+'F1scoreAllRecords.csv', mode='a+',hearder=False)
bestTh = max(result.items(), key=operator.itemgetter(1))[0]

#use best th to calculate again
outliers = anomalyDetection(matrix,pval = bestTh)
df['outlier'] = outliers
TP = len(df[(df['outlier']==True)&(df['Anomaly']==True)])
FP = len(df[(df['outlier']==False)&(df['Anomaly']==True)])
TN = len(df[(df['outlier']==False)&(df['Anomaly']==False)])
FN = len(df[(df['outlier']==False)&(df['Anomaly']==True)])

f.wrtie('TP: '+str(TP)+' FP: '+str(FP)+' TN: '+str(TN)+' FN: '+str(FN)+'\n')
f.write('overall F1: '+str(result[bestTh])+', Threshold: '+str(bestTh)+'\n')
f.write('Events amount: '+str(len(EventDFTrans))+' Days: '+str(len(datewiseDF))+' ratio: '+str(len(EventDFTrans)/len(datewiseDF)))
EventType = list(df_finalEvents['Type'].unique())
f.close()

nodes amount:  108
communities numbers:  ['3', '1', '0', '2']
detecting sub-communities in community  3
nodes amount:  21
detecting sub-communities in community  1
nodes amount:  24
detecting sub-communities in community  0
nodes amount:  23
detecting sub-communities in community  2
nodes amount:  40
Raw shape:  (116640000, 5)
